In [7]:
# Import necessary libraries
import re
import os
import json
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [2]:
# Load the config file to get the ontology categories
config_path = 'config/dbpedia_webnlg_prompt_gen_config.json'
with open(config_path, 'r') as f:
    config = json.load(f)
ontology_list = config['onto_list']

In [3]:
# Initialize the SentenceTransformer model
# Note: T5-XXL is a very large model (11B parameters) and may not run on standard hardware.
# Make sure you have the necessary resources before loading this model.
model_name = 'sentence-t5-xxl'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:127: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/pytorch/pytorch/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
# Define paths
test_data_path = '../../data/dbpedia_webnlg/test/'
train_data_path = '../../data/dbpedia_webnlg/train/'
output_path = '../../data/dbpedia_webnlg/baselines/test_train_sent_similarity/'

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

In [8]:
# Number of top similar sentences to retrieve
top_k = 5

# Process each ontology category
for ontology in ontology_list:
    print(f'Processing ontology: {ontology}')
    
    # Extract the number from the ontology string
    ontology_num = re.search(r'(\d+)_', ontology).group(1)
    category = ontology.split('_', 1)[1]  # Get category name after number
    
    # Load test data
    test_file = os.path.join(test_data_path, f'ont_{ontology_num}_{category}_test.jsonl')
    test_sentences = []
    test_ids = []
    with open(test_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line.strip())
            test_sentences.append(data['sent'])
            test_ids.append(data['id'])

    # Load train data
    train_file = os.path.join(train_data_path, f'ont_{ontology_num}_{category}_train.jsonl')
    train_sentences = []
    train_ids = []
    with open(train_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line.strip())
            train_sentences.append(data['sent'])
            train_ids.append(data['id'])

    # Compute embeddings for test and train sentences
    print('Computing embeddings for test sentences...')
    test_embeddings = model.encode(test_sentences, convert_to_tensor=True, show_progress_bar=True)
    print('Computing embeddings for train sentences...')
    train_embeddings = model.encode(train_sentences, convert_to_tensor=True, show_progress_bar=True)

    # Compute similarities and find top-k similar sentences for each test sentence
    similarity_results = {}
    print('Computing similarities and finding top similar sentences...')
    for idx, test_embedding in enumerate(tqdm(test_embeddings)):
        cosine_scores = util.cos_sim(test_embedding, train_embeddings)[0]
        top_results = torch.topk(cosine_scores, k=top_k)
        similar_train_ids = [train_ids[i] for i in top_results[1]]
        similarity_results[test_ids[idx]] = similar_train_ids

    # Save the results to the output file
    output_file = os.path.join(output_path, f'{ontology}_test_train_similarity.json')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(similarity_results, f, indent=4)

    print(f'Results saved to {output_file}\n')

print('Processing completed.')

Processing ontology: 10_city
Computing embeddings for test sentences...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches: 100%|██████████| 7/7 [01:41<00:00, 14.53s/it]


Computing embeddings for train sentences...


Batches: 100%|██████████| 5/5 [01:23<00:00, 16.78s/it]


...

Computing embeddings for train sentences...


Batches: 100%|██████████| 6/6 [01:43<00:00, 17.19s/it]


Computing similarities and finding top similar sentences...


100%|██████████| 103/103 [00:00<00:00, 4642.46it/s]

Results saved to ../../data/dbpedia_webnlg/baselines/test_train_sent_similarity/9_building_test_train_similarity.json

Processing completed.
